<div style="text-align: center;">
  <h2>Fine Tuning Models</h2>
</div>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

data = pd.read_csv("clean_data.csv")
data.head()

,Unnamed: 0,Label,Text
0,0,Politics,budget to set scene for election gordon brown ...
1,1,Politics,army chiefs in regiments decision military chi...
2,3,Politics,observers to monitor uk election ministers wil...
3,4,Politics,kilroy names election seat target exchat show ...
4,5,Politics,donor attacks blairbrown feud the reported feu...


In [2]:
data.replace({0:"Politics", 1:"Sport", 2:"Technology", 3:"Entertainment", 4:"Business"}, inplace=True)
data.Label.value_counts()

Label
Sport            505
Business         503
Politics         403
Entertainment    369
Technology       347
Name: count, dtype: int64

In [3]:
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'])

In [4]:
X_train, X_val, y_train, y_val = train_test_split(data["Text"], data["Label"], test_size=0.2, random_state=42)

In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(X_val), truncation=True, padding=True, max_length=128)

- Defines a class called **`SentimentDataset`** derived from PyTorch's **`torch.utils.data.Dataset`** class. This class will include the necessary functions to access each item in the dataset and determine its size.
- The **`__init__`** function is executed when an instance of the class is created.
- **`encodings`**: This parameter represents the tokenized version of the text data (e.g., obtained using BERT tokenization like `train_encodings`).
- **`labels`**: This parameter contains the sentiment labels of the text data (e.g., positive, negative, or neutral labels).
- The **`__getitem__`** function allows retrieving an item from the dataset using a specific **`idx`** (index).
- **`self.encodings.items()`**: Retrieves each item from the tokenized texts and converts each one into a PyTorch tensor with **`torch.tensor(val[idx])`**.
- **`item['labels'] = torch.tensor(int(self.labels[idx]))`**: In this line, the label of the text is retrieved and converted to an integer before being added to the **`labels`** key.
- The **`__len__`** function returns the size of the dataset. This gives the number of items in the dataset (e.g., equal to the number of labels).

In [6]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(int(self.labels[idx])) 
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, y_train.tolist())
val_dataset = SentimentDataset(val_encodings, y_val.tolist())

- `BertForSequenceClassification.from_pretrained('bert-base-uncased')`: Bu fonksiyon, BERT modelini 'bert-base-uncased' olarak yükler. Bu model, Wikipedia verisinde eğitilmiş ve case-sensitive (büyük/küçük harf ayrımı yapmayan) bir modeldir. Yani, "Apple" ve "apple" gibi kelimeleri aynı şekilde ele alır.

- `num_labels=5`: Bu parametre, modelin çıktısının 5 farklı etiketten oluşacağını belirtir. Bu, modelin her metin için 5 farklı olası sınıf tahmini yapacağı anlamına gelir.

In [7]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5) 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


- **`TrainingArguments`** : Bu sınıf, modelin eğitilmesi için gerekli olan tüm parametreleri içerir.
- **`output_dir='./results'`** : Eğitim sonuçlarının (model, günlükler vb.) saklanacağı dizini belirtir.
- **`eval_strategy="epoch"`** : Modelin ne zaman değerlendirileceğini belirtir. Burada, değerlendirme her bir **epoch** (tam eğitim geçişi) sonunda yapılacaktır.
- **`per_device_train_batch_size=8`** : Her bir cihazda (CPU/GPU) eğitim için kullanılan örnek sayısını belirtir. Bu örnekte her batch (grup) 8 örnekten oluşacaktır.
- **`per_device_eval_batch_size=8`** : Değerlendirme sırasında her cihazda kullanılan batch boyutunu belirtir. Burada da batch boyutu 8'dir.
- **`num_train_epochs=3`** : Modelin eğitileceği epoch sayısını belirtir. Bu durumda model 3 epoch boyunca eğitilecektir.
- **`weight_decay=0.01`** : **Ağırlık çürütme (weight decay)** parametresi, aşırı öğrenmeyi (overfitting) engellemek için kullanılan bir regularization tekniğidir. Modelin her parametresi, her güncellemede %1 (0.01) kadar azalacaktır.

---
- **`compute_metrics`** : Bu fonksiyon, eğitim ve değerlendirme sırasında modellerin performansını ölçmek için kullanılan metrikleri hesaplar (bu örnekte **doğruluk**).
- **`pred.label_ids`** : Gerçek etiketler (hedef değerler) olan doğru etiketler.
- **`pred.predictions`** : Modelin tahminleri.
- **`np.argmax(pred.predictions, axis=1)`** : Modelin yaptığı tahminlerde, her örnek için en yüksek olasılığa sahip sınıfın indeksini alır.
- **`accuracy_score(labels, preds)`** : Gerçek etiketler ile modelin tahminleri arasındaki doğruluğu hesaplar.
---
- **`Trainer`** : **Trainer** sınıfı, Hugging Face tarafından sağlanan ve modelin eğitilmesini yöneten bir sınıftır.
- **`model=model`** : Eğitilecek model (bu durumda yüklü olan BERT modeli).
- **`args=training_args`** : Önceden yapılandırılmış eğitim parametreleri (batch boyutu, epoch sayısı, vb.).
- **`train_dataset=train_dataset`** : Modelin eğitim için kullanacağı veri seti.
- **`eval_dataset=val_dataset`** : Modelin her epoch sonunda performansını değerlendireceği doğrulama veri seti.
- **`compute_metrics=compute_metrics`** : Eğitim ve değerlendirme sırasında kullanılacak metrik hesaplama fonksiyonu (burada doğruluk).

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [9]:
trainer.train()
eval_results = trainer.evaluate()
print(f"Accuracy: {eval_results['eval_accuracy']}")
print(f"Loss: {eval_results['eval_loss']}")

  0%|          | 0/639 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

{'eval_loss': 0.07627822458744049, 'eval_accuracy': 0.9859154929577465, 'eval_runtime': 53.5089, 'eval_samples_per_second': 7.961, 'eval_steps_per_second': 1.009, 'epoch': 1.0}


  0%|          | 0/54 [00:00<?, ?it/s]

{'eval_loss': 0.12288319319486618, 'eval_accuracy': 0.9788732394366197, 'eval_runtime': 51.1887, 'eval_samples_per_second': 8.322, 'eval_steps_per_second': 1.055, 'epoch': 2.0}
{'loss': 0.1486, 'grad_norm': 0.014019527472555637, 'learning_rate': 1.0876369327073553e-05, 'epoch': 2.35}


  0%|          | 0/54 [00:00<?, ?it/s]

{'eval_loss': 0.09346222877502441, 'eval_accuracy': 0.9812206572769953, 'eval_runtime': 52.8672, 'eval_samples_per_second': 8.058, 'eval_steps_per_second': 1.021, 'epoch': 3.0}
{'train_runtime': 2700.3861, 'train_samples_per_second': 1.89, 'train_steps_per_second': 0.237, 'train_loss': 0.12089223406497675, 'epoch': 3.0}


  0%|          | 0/54 [00:00<?, ?it/s]

Accuracy: 0.9812206572769953
Loss: 0.09346222877502441


In [10]:
test_text = ["The government has announced a new policy aimed at reducing carbon emissions by 50% by 2030."]
test_encodings = tokenizer(test_text, truncation=True, padding=True, max_length=128, return_tensors="pt")
outputs = model(**test_encodings)
preds = torch.argmax(outputs.logits, dim=1)
predicted_label = label_encoder.inverse_transform(preds.numpy())
print(f"Predicted label: {predicted_label[0]}")

Predicted label: Politics


In [11]:
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\vocab.txt',
 './saved_model\\added_tokens.json')

In [12]:
import gradio as gr
model = BertForSequenceClassification.from_pretrained('./saved_model')
tokenizer = BertTokenizer.from_pretrained('./saved_model')

- `RoBERTa`, 2019 yılında Facebook tarafından geliştirilen ve BERT’in geliştirilmiş bir versiyonudur. BERT ile aynı mimariye sahiptir ancak bazı eğitim süreci optimizasyonları yapılmıştır: RoBERTa, daha fazla veri üzerinde, daha uzun sürelerde ve BERT'teki bazı sınırlamaların (örneğin, sonraki cümle tahmini gibi) kaldırılmasıyla eğitilmiştir. Bu iyileştirmeler sayesinde RoBERTa, karmaşık ilişkileri daha iyi öğrenir ve birçok NLP görevinde, özellikle metin sınıflandırma gibi alanlarda daha yüksek performans gösterir.

In [15]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
train_encodings_roberta = roberta_tokenizer(list(X_train), truncation=True, padding=True, max_length=128)
val_encodings_roberta = roberta_tokenizer(list(X_val), truncation=True, padding=True, max_length=128)

In [17]:
train_dataset_roberta = SentimentDataset(train_encodings_roberta, y_train.tolist())
val_dataset_roberta = SentimentDataset(val_encodings_roberta, y_val.tolist())

In [18]:
training_args_roberta = TrainingArguments(
    output_dir='./results_roberta',
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [19]:
trainer_roberta = Trainer(
    model=roberta_model,
    args=training_args_roberta,
    train_dataset=train_dataset_roberta,
    eval_dataset=val_dataset_roberta,
    compute_metrics=compute_metrics
)

In [20]:
trainer_roberta.train()

eval_results_roberta = trainer_roberta.evaluate()
print(f"RoBERTa Accuracy: {eval_results_roberta['eval_accuracy']}")
print(f"RoBERTa Loss: {eval_results_roberta['eval_loss']}")

  0%|          | 0/639 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

{'eval_loss': 0.12714247405529022, 'eval_accuracy': 0.9765258215962441, 'eval_runtime': 46.846, 'eval_samples_per_second': 9.094, 'eval_steps_per_second': 1.153, 'epoch': 1.0}


  0%|          | 0/54 [00:00<?, ?it/s]

{'eval_loss': 0.1554717719554901, 'eval_accuracy': 0.9788732394366197, 'eval_runtime': 45.2472, 'eval_samples_per_second': 9.415, 'eval_steps_per_second': 1.193, 'epoch': 2.0}
{'loss': 0.2, 'grad_norm': 0.013453349471092224, 'learning_rate': 1.0876369327073553e-05, 'epoch': 2.35}


  0%|          | 0/54 [00:00<?, ?it/s]

{'eval_loss': 0.1385667771100998, 'eval_accuracy': 0.9765258215962441, 'eval_runtime': 43.9853, 'eval_samples_per_second': 9.685, 'eval_steps_per_second': 1.228, 'epoch': 3.0}
{'train_runtime': 2472.2344, 'train_samples_per_second': 2.064, 'train_steps_per_second': 0.258, 'train_loss': 0.16818640265666263, 'epoch': 3.0}


  0%|          | 0/54 [00:00<?, ?it/s]

RoBERTa Accuracy: 0.9765258215962441
RoBERTa Loss: 0.1385667771100998


In [21]:
roberta_model.save_pretrained('./saved_model_roberta')
roberta_tokenizer.save_pretrained('./saved_model_roberta')

('./saved_model_roberta\\tokenizer_config.json',
 './saved_model_roberta\\special_tokens_map.json',
 './saved_model_roberta\\vocab.json',
 './saved_model_roberta\\merges.txt',
 './saved_model_roberta\\added_tokens.json')

In [22]:
def predict_with_choice(text, model_choice):
    if model_choice == "BERT":
        # Use BERT for prediction
        inputs = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="pt")
        outputs = model(**inputs)
    elif model_choice == "RoBERTa":
        # Use RoBERTa for prediction
        inputs = roberta_tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="pt")
        outputs = roberta_model(**inputs)

    preds = torch.argmax(outputs.logits, dim=1)
    predicted_label = label_encoder.inverse_transform(preds.numpy())
    return predicted_label[0]


In [23]:
iface = gr.Interface(
    fn=predict_with_choice,
    inputs=[
        gr.Textbox(label="Text to Classify"),  
        gr.Dropdown(choices=["BERT", "RoBERTa"], label="Choose Model"),  
    ],
    outputs="text",
    title="Text Classification with BERT and RoBERTa",
    description="Enter a text and select a model (BERT or RoBERTa) to predict its category."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
